In [8]:
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
from langchain_huggingface import HuggingFaceEndpoint

In [10]:
import os
repo_id="mistralai/Mistral-7B-Instruct-v0.3"
llm=HuggingFaceEndpoint(repo_id=repo_id,max_length=150,temperature=0.7,token=os.getenv("HF_TOKEN"))
poem = llm("Write a 4 line poem on my love for falafel")
print(poem)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.
C:\Users\Uyama\AppData\Local\Temp\ipykernel_19664\1679390358.py:4: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  poem = llm("Write a 4 line poem on my love for falafel")




In the heart of the Middle East, a ball of delight,
Falafel, you dance in pita's soft light.
A symphony of spices, a burst of flavor,
My love for you, will forever endure.


In [11]:
essay = llm("write an email requesting refund for electronic item")
print(essay)



Subject: Request for Refund for Faulty [Product Name]

Dear [Customer Service Representative's Name],

I hope this email finds you well. I am writing to request a refund for the [Product Name] that I purchased on [Date of Purchase] from [Your Store's Name].

Upon receiving the product, I discovered that it was faulty. Specifically, [describe the issue in detail, for example, "the screen is cracked", "the device does not power on", "the product is missing parts", etc.]. I have attached photographs of the issue for your reference.

I have tried to troubleshoot the problem, but unfortunately, I have been unable to resolve it. I have also reached out to your technical support team, but they were unable to assist me.

I would like to request a refund for the product as it is not fit for use. I have used the product for a short period of time, and I believe that a refund would be the fairest resolution in this situation.

I have included my order number [Order Number] and my contact detail

In [1]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader("tech_faqs.csv", encoding='ISO-8859-1')
data=loader.load()
len(data)

75

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
e = embeddings.embed_query("What is your refund policy?")

c:\Users\Uyama\Downloads\LANGCHAIN-PROJECTS\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Uyama\Downloads\LANGCHAIN-PROJECTS\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
len(e)

384

In [4]:
e[:5]

[-0.07025409489870071,
 0.04848405718803406,
 0.048615992069244385,
 0.018366418778896332,
 0.05777846276760101]

In [5]:
from langchain.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [6]:
rdocs = retriever.get_relevant_documents("how about job placement support?")
rdocs

C:\Users\Uyama\AppData\Local\Temp\ipykernel_19664\3956196717.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  rdocs = retriever.get_relevant_documents("how about job placement support?")


[Document(metadata={'source': 'tech_faqs.csv', 'row': 11}, page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.'),
 Document(metadata={'source': 'tech_faqs.csv', 'row': 33}, page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.'),
 D

### Create RetrievalQA chain along with prompt template

In [12]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [13]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [14]:
chain('Do you provide job assistance and also do you provide job guarantee?')

C:\Users\Uyama\AppData\Local\Temp\ipykernel_19664\470731276.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  chain('Do you provide job assistance and also do you provide job guarantee?')


{'query': 'Do you provide job assistance and also do you provide job guarantee?',
 'result': '\nANSWER: Yes, we provide job assistance. However, we do not guarantee a job as we want to be honest and do not want to make any impractical promises. Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.',
 'source_documents': [Document(metadata={'source': 'tech_faqs.csv', 'row': 11}, page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.'),
  Document(metadata={'source': 'tech_faqs.csv', 'row': 33}, page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that 

In [15]:
chain("Do you guys provide internship and also do you offer EMI payments?")

{'query': 'Do you guys provide internship and also do you offer EMI payments?',
 'result': "\n\nANSWER: Yes, we provide virtual internships. However, I don't have information about EMI payment options.",
 'source_documents': [Document(metadata={'source': 'tech_faqs.csv', 'row': 14}, page_content='prompt: Do you provide any virtual internship?\nresponse: Yes'),
  Document(metadata={'source': 'tech_faqs.csv', 'row': 13}, page_content='prompt: Do we have an EMI option?\nresponse: No'),
  Document(metadata={'source': 'tech_faqs.csv', 'row': 11}, page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.'),
  Document(metadata={'source': 'tech_faqs.csv', 'row': 22}, page_content='prompt: Once purchased, is this course available for lifetime access?\nresponse: Yes')]}

In [16]:
chain("do you have javascript course?")

{'query': 'do you have javascript course?',
 'result': "\nANSWER: I don't know. The context does not provide information about any JavaScript course.",
 'source_documents': [Document(metadata={'source': 'tech_faqs.csv', 'row': 18}, page_content='prompt: What is different in this course compared to hundreds of courses on the internet and free tutorials on YouTube?\nresponse: Most of the courses available on the internet teach you how to build x & y without any business context and do not prepare you for real business world problem-solving. This course is rather an experience in which you will learn Excel by solving real-life use cases in an imaginary company called AtliQ Hardware. The tutorials are very easy to understand and also have a lot of fun elements into them so that you don\x92t get bored ??'),
  Document(metadata={'source': 'tech_faqs.csv', 'row': 25}, page_content='prompt: I don\x92t have a laptop, can I take this course?\nresponse: We recommend learning by doing and therefor

In [17]:
chain("Do you have plans to launch blockchain course in future?")

{'query': 'Do you have plans to launch blockchain course in future?',
 'result': "\nresponse: At this moment, we don't have plans for a blockchain course. Our focus is on the consumer goods domain and the core functions of a business as mentioned earlier. We believe that the consumer goods domain will have more openings and high data analytics requirements at least until 2030.",
 'source_documents': [Document(metadata={'source': 'tech_faqs.csv', 'row': 32}, page_content='prompt: What business concepts and domains are covered in this course?\nresponse: We have covered the core functions such as Sales, Marketing, Finance, and Supply Chain with their fundamentals related to this course. The domain you will learn in this course is consumer goods which is projected to have more openings and high data analytics requirements at least until 2030.'),
  Document(metadata={'source': 'tech_faqs.csv', 'row': 28}, page_content='prompt: I use tableau, can I take this course?\nresponse: Yes, you will 

In [18]:
chain("should I learn power bi or tableau?")

{'query': 'should I learn power bi or tableau?',
 'result': '\n\nANSWER: The choice between Power BI and Tableau depends on the context. If you are talking about a pure visualization tool, Tableau is slightly better. However, if you consider data connectors, modeling and transformation features, both Power BI and Tableau offer similar capabilities. Additionally, Power BI is cheaper and offers tighter integration with the Microsoft environment, making it a popular choice for companies using Microsoft tools. Since most companies use excel & Microsoft tools, they often start with Power BI or move towards Power BI for seamless integration with other Microsoft tools (called as Power platform). This makes the job openings grow at a much higher rate on Power BI and Power Platform. Therefore, it might be more beneficial to learn Power BI if you are working in a Microsoft-centric environment.',
 'source_documents': [Document(metadata={'source': 'tech_faqs.csv', 'row': 29}, page_content='prompt:

In [19]:
chain("I've a MAC computer. Can I use powerbi on it?")

{'query': "I've a MAC computer. Can I use powerbi on it?",
 'result': '\nANSWER: Yes, you can use Power BI on your Mac by creating a virtual machine and installing Windows on it.',
 'source_documents': [Document(metadata={'source': 'tech_faqs.csv', 'row': 44}, page_content='prompt: How can I use PowerBI on my Mac system?\nresponse: Hi\n\nYou can use VirtualBox to create a virtual machine and install Windows on it. This will allow you to run Power BI and Excel on your Mac.\n\nIf you\'re not familiar with setting up a virtual machine, there are many resources available on YouTube that can guide you through the process. Simply search for "installing virtual machines" and you\'ll find plenty of helpful videos.\n\nBest of luck with your studies!'),
  Document(metadata={'source': 'tech_faqs.csv', 'row': 31}, page_content='prompt: Does Power BI work in Mac OS/Ubuntu?\nresponse: Power BI desktop works only in Windows OS. Please look into the system requirements section on this page. However, y

In [20]:
chain("I don't see power pivot. how can I enable it?")

{'query': "I don't see power pivot. how can I enable it?",
 'result': '\n\nANSWER: Follow the process provided in this link - https://drive.google.com/file/d/1-mO-v52h-YTY1s-v30liBJPu6Yj4OUxb/view?usp=share_link',
 'source_documents': [Document(metadata={'source': 'tech_faqs.csv', 'row': 74}, page_content='prompt: How do I enable Power Pivot before using it for the first time ?\nresponse: Follow the process in the link : \n\nhttps://drive.google.com/file/d/1-mO-v52h-YTY1s-v30liBJPu6Yj4OUxb/view?usp=share_link'),
  Document(metadata={'source': 'tech_faqs.csv', 'row': 38}, page_content='prompt: How to install power pivot if its not available in system?\nresponse: Follow this thread for instructions - https://support.microsoft.com/en-us/office/start-the-power-pivot-add-in-for-excel-a891a66d-36e3-43fc-81e8-fc4798f39ea8\nIf it doesn\'t show in the ribbon then go to "insert" tab. You will be able to see pivot table option there.'),
  Document(metadata={'source': 'tech_faqs.csv', 'row': 46}, 

In [21]:
chain("What is the price of your machine learning course?")

{'query': 'What is the price of your machine learning course?',
 'result': "\n\nANSWER: I don't know. The context does not provide any information about the price of the machine learning course.",
 'source_documents': [Document(metadata={'source': 'tech_faqs.csv', 'row': 35}, page_content='prompt: Is there any prerequisite for taking this course?\nresponse: The only prerequisite is that you need to have a functional laptop with at least 4GB ram, internet connection and a thrill to learn data analysis.'),
  Document(metadata={'source': 'tech_faqs.csv', 'row': 25}, page_content='prompt: I don\x92t have a laptop, can I take this course?\nresponse: We recommend learning by doing and therefore you need to have a laptop or a PC (at least 4 GB ram).'),
  Document(metadata={'source': 'tech_faqs.csv', 'row': 20}, page_content='prompt: I\x92m not sure if this course is good enough for me to invest some money. What can I do?\nresponse: Don\x92t worry. Many videos in this course are free so watch 